# Código Postal

### Bibliotecas

- **brazilcep:** para descobrir o Endereço pelo CEP.
- **Geopy:** Transforma o Endereço em Coordenadas.

In [ ]:
import brazilcep
from geopy.geocoders import Nominatim

- Solicita o CEP e transforma em endereço:

In [ ]:
cep = str(input('Informe o CEP do Inicio (Sem Traços): ')).strip()
endereco = brazilcep.get_address_from_cep(cep)
print(endereco)
geolocator = Nominatim(user_agent="test_app")
location = geolocator.geocode(endereco['street'] + ", " + endereco['city'])


- A partir do endereço descoberto, é extraído suas coordenadas.

In [ ]:
print(location.latitude)
latitude = location.latitude
longitude = location.longitude

- Assim como o Endereço Inicial, também é solicitado o CEP para o Endereço de Chegada, com suas coordenadads salvas em outras variáveis

In [ ]:
cep = str(input('Informe o CEP do local a chegar (Sem Traços): ')).strip()
endereco = brazilcep.get_address_from_cep(cep)
geolocator = Nominatim(user_agent="test_app")
location = geolocator.geocode(endereco['street'] + ", " + endereco['city'])

latitude2 = location.latitude
longitude2 = location.longitude

# Folium (Mapa)

### Bibliotecas

- **Folium:** Para mostrar o mapa
- **Openrouteservice:** Mostra a rota para o lugar determinado

### Código

- inicio e chegada recebem latitudes e longitudes;
- a variável m recebe a localização no mapa;
- Marcadores são adicionados nas localizações informadas;
- Rota recebe devidas informações;
- Com essas informações, é criada a linha no mapa.

In [ ]:
import folium
import openrouteservice as ors

client = ors.Client(key='5b3ce3597851110001cf6248c92503e4c87f494aa259b8682754547b')

inicio = float(latitude), float(longitude)
chegada = float(latitude2), float(longitude2)
coords = [[float(longitude), float(latitude)], [float(longitude2), float(latitude2)]]

m = folium.Map(location=inicio, zoom_start=13)

folium.Marker(inicio, popup='Inicio!', icon=folium.Icon(color="red")).add_to(m)
folium.Marker(chegada, popup='Chegada!').add_to(m)


route = client.directions(coordinates=coords,
                          profile='driving-car',
                          format='geojson')


folium.PolyLine(locations=[list(reversed(coord)) for coord in route['features'][0]['geometry']['coordinates']], color="blue").add_to(m)

m



# Várias Entregas

No caso de várias entregas, estas serão distribuidas entre os veículos de acordo com suas capacidades.

### Código

Primeiro, você criarará uma lista com coordenadas que serão os pontos de entregas

Você pode criar a lista, tanto:

- Usando o gerador de coordenas com base no CEP do Local de Entrega
- Digitando as coordenas diretamente na lista -> `coords` (Lembrando de escrever em ordem invertida (Longitude, Latitude))

---

- A variável Inicio guarda a coordenada do Ponto de Partida;
- É criado o mapa e os Locais de Entrega e o Ponto de Partida são marcados no mesmo;
- Na Lista `vehicles` são salvos os veículos e a quantidade de cada;
- É usado a função de jobs e optimized, para criar várias rotas otimizadas para os serviços;
- São usados **cores** para **diferenciar** os carros;
- As Rotas são adicionadas ao mapa.


In [ ]:
# Lista pré-gerada

coords = [
    [-46.68964,-23.74480],
    [-46.72016,-23.60005],
    [-46.71535,-23.59984],
    [-46.63278,-23.52955],
    [-46.65597,-23.56150],
    [-46.68216,-23.74369],
    [-46.6407,-23.5453],
]

# Gerar a lista de Locais de Entrega
'''
r = 's'
while r == 's' or r == 'sim':
    cep = str(input('Informe o CEP do Local de Entrega (Sem Traços): ')).strip()
    endereco = brazilcep.get_address_from_cep(cep, timeout=10)
    geolocator = Nominatim(user_agent="test_app")
    location = geolocator.geocode(endereco['street'] + ", " + endereco['city'])
    coords.append([location.longitude, location.latitude])
    r = input('Deseja adicionar mais um local de entrega? (s/n): ').lower().strip()
'''

inicio = [-46.72924, -23.66245]

m = folium.Map(location=list(reversed(inicio)), tiles="cartodbpositron", zoom_start=14)
for coord in coords:
    folium.Marker(location=list(reversed(coord)), popup='Local de Entrega').add_to(m)
    
folium.Marker(location=list(reversed(inicio)), popup='Ponto de Partida', icon=folium.Icon(color="red")).add_to(m)

vehicles = [
    ors.optimization.Vehicle(id=0, profile='driving-car', start=inicio, end=inicio, capacity=[5]),
    ors.optimization.Vehicle(id=1, profile='driving-car', start=inicio, end=inicio, capacity=[5])
]
jobs = [ors.optimization.Job(id=index, location=coords, amount=[1]) for index, coords in enumerate(coords)]
optimized = client.optimization(jobs=jobs, vehicles=vehicles, geometry=True)

line_colors = ['green', 'orange', 'blue', 'yellow']

for route in optimized['routes']:
    folium.PolyLine(locations=[list(reversed(coords)) for coords in ors.convert.decode_polyline(route['geometry'])['coordinates']], color=line_colors[route['vehicle']]).add_to(m)
    

m
